# Определение языка и VK API

В данном задании вам нужно будет:

* используя API Вконтакте, скачать комментарии к первым ста постам из пяти сообществ
* натренировать модель распознавания языков на статьях из Википедии.
* распознать язык всех комментариев, где в тексте есть 10 и более символов, и построить статистику

## VK API

Для подключения к ВКонтакте мы будем использовать VK API. Здесь есть документация к этой библиотеке https://vk-api.readthedocs.io/en/latest/

In [ ]:
!pip3 install vk_api

In [ ]:
import vk_api

# здесь нужно ввести данные своего аккаунта
# когда будете сдавать, не забудьте убрать эти две строчки из ноутбука
user = '' # вставьте сюда свой номер телефона
password = '' # вставьте сюда свой пароль

# авторизация
vk_session = vk_api.VkApi(login=user, password=password)
vk_session.auth()

vk = vk_session.get_api() # объект с API

Получить записи со страницы можно с помощью метода `wall.get`. Он принимает параметр `domain` — короткое имя пользователя или сообщества — и `count` — количество записей, которое вы хотите получить (максимум — 100). По Список методов для работы со стенами: https://vk.com/dev/wall

Например, вот так можно получить последние две записи с вот этой страницы https://vk.com/futureisnow. Выдача представляет собой словарь, в котором в поле `items` записан список словарей, содержащий информацию о каждой из записи. 

In [ ]:
vk.wall.get(domain="futureisnow", count=2)

С помощью метода `groups.getById` можно получить информацию о сообществе, в том числе его id.

In [ ]:
vk.groups.getById(group_ids="futureisnow")

Скачайте информацию о последних ста записях в следующих пабликах: https://vk.com/futureisnow, https://vk.com/eternalclassic, https://vk.com/ukrlit_memes, https://vk.com/ukrainer_net, https://vk.com/amanzohel, https://vk.com/barg_kurumk_culture

In [ ]:
# список domain'ов, чтобы вам не копировать их самими :)
publics = ["futureisnow",
           "eternalclassic",
           "ukrlit_memes",
           "ukrainer_net",
           "amanzohel",
           "barg_kurumk_culture"]

In [ ]:
publics_ids = {}

for i in vk.groups.getById(group_ids=publics):
    publics_ids[i['screen_name']] = -i['id']

In [ ]:
publics = publics_ids

In [ ]:
publics

In [ ]:
items = {} # ключи — это паблики

for i in publics.keys():
    all_info = vk.wall.get(domain=i, count=100)
    items[i] = all_info['items']

In [ ]:
all_posts = []

for i in items:
    for each_post in items[i]:
        all_posts.append([each_post['owner_id'], each_post['id']])

Найдите в документации (https://vk.com/dev/wall) метод для получения комментариев и получите первые сто комментариев каждого поста из выборки для каждого паблика.

In [ ]:
infos = {}

for i in all_posts:
    infos[i[0], i[1]] = vk.wall.getComments(owner_id=i[0],
                                            post_id=i[1], count=100)

In [ ]:
corpora = {}

fut = []
ete = []
ukm = []
ukn = []
ama = []
barg = []

for i in infos:
    for j in infos[i]['items']:
        try:
            if len(j['text']) > 10:
                if j['owner_id'] == publics['futureisnow']:
                    fut.append(j['text'])
                elif j['owner_id'] == publics['eternalclassic']:
                    ete.append(j['text'])
                elif j['owner_id'] == publics['ukrlit_memes']:
                    ukm.append(j['text'])
                elif j['owner_id'] == publics['ukrainer_net']:
                    ukn.append(j['text'])
                elif j['owner_id'] == publics['amanzohel']:
                    ama.append(j['text'])
                elif j['owner_id'] == publics['barg_kurumk_culture']:
                    barg.append(j['text'])
        except:
            continue
corpora['futureisnow'] = fut
corpora['eternalclassic'] = ete
corpora['ukrlit_memes'] = ukm
corpora['ukrainer_net'] = ukn
corpora['amanzohel'] = ama
corpora['barg_kurumk_culture'] = barg

In [ ]:
punc = '!@#$%^&*()_—-=+~:;`\ "\'/?«».>,<'

clear_corpora = {}

for i in corpora:
    clear_comments = []
    for j in corpora[i]:
        clear_comment = []
        parts = nltk.word_tokenize(j.lower())
        clear_parts = []
        for part in parts:
            clear_parts.append(part.strip(punc))
        clear_comment = ' '.join(clear_parts)
        if len(clear_comment) > 15:
            clear_comments.append(clear_comment.strip())
    clear_corpora[i] = clear_comments

**Задание со звездочкой:** вы могли заметить, что если обращаться к каждому посту отдельно, то все занимает довольно продолжительное время (около пяти минут). Найдите в документации vk_api способ сделать это быстрее.

In [ ]:
corpora = {}

# ваш улучшенный код

## Тренировка моделей

В наших комментариях встречались русский, украинский, английский и бурятский.

In [ ]:
langs = {'ru', 'uk', 'en', 'bxr'}

In [ ]:
from collections import Counter

import nltk
import wikipedia

In [ ]:
nltk.download("punkt")

Скачайте документы, на которых вы будете обучать свои модели. Для наших целей хорошо иметь для каждого языка корпус размером около 50 статей.

In [ ]:
def get_texts_for_lang(lang):
    wiki_content = []
    wikipedia.set_lang(lang)
    pages = wikipedia.random(60)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
            wiki_content.append("%s\n%s" % (page.title, 
                                            page.content.replace('=', '')))
        except wikipedia.exceptions.WikipediaException:
            print("Skip %s" % page_name)
    return wiki_content

In [ ]:
wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang)
    print(lang, len(wiki_texts[lang]))

Сделайте определялку на частотах слов.

In [ ]:
def collect_freqlist(wiki_pages, max_len=100):
    freqlist = Counter()
    for text in wiki_pages:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha():
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

In [ ]:
freq_lists = {}
for lang in langs:
    freq_lists[lang] = collect_freqlist(wiki_texts[lang])

In [ ]:
def simple_lang_detect(freq_lists, text):
    counts = Counter()
    for lang, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[lang] += int(freq_list[word] > 0)
    return counts.most_common()

Сделайте определялку на символьных энграммах.

In [ ]:
from sklearn import feature_extraction

vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1, 5),
                                                     analyzer='char')
vectorizer.fit(wiki_texts['ru'])
for item in vectorizer.get_feature_names()[:100]:
    print(item)

In [ ]:
from sklearn import pipeline
from sklearn import naive_bayes
import numpy as np

%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
clf = pipeline.Pipeline([
    ('vctr', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2),
                                                     analyzer='char')),
    ('clf', naive_bayes.MultinomialNB())
])

In [ ]:
all_texts = []
lang_indices = []
for lang in wiki_texts:
    all_texts.extend(wiki_texts[lang])
    lang_indices.extend([lang]*len(wiki_texts[lang]))

In [ ]:
clf.fit(np.array(all_texts), np.array(lang_indices))

# Определение языка

Определите язык каждого комментария в каждом паблике с помощью определялки на частотах слов и покажите доли языков среди комментариев для каждого паблика.

In [ ]:
lang_detects_freqs = {}

for i in clear_corpora:
    new_list = []
    for j in clear_corpora[i]:
        try:
            result = simple_lang_detect(freq_lists, j)
            if result[0][1] > 0:
                new_list.append(result[0][0])
            else:
                new_list.append('err')
        except IndexError:
            continue
    lang_detects_freqs[i] = Counter(new_list)

In [ ]:
ldf = {}

for a, b in lang_detects_freqs.items():
    ldf[a] = {}
    
for key, value in lang_detects_freqs.items():
    if 'ru' in value:
        ldf[key]['русский'] = lang_detects_freqs[key]['ru']
    if 'uk' in value:
        ldf[key]['украинский'] = lang_detects_freqs[key]['uk']
    if 'en' in value:
        ldf[key]['английский'] = lang_detects_freqs[key]['en']
    if 'err' in value:
        ldf[key]['ошибка'] = lang_detects_freqs[key]['err']
    if 'bxr' in value:
        ldf[key]['бурятский'] = lang_detects_freqs[key]['bxr']

In [ ]:
for i in ldf:
    print('\nВ паблике ', i, ':', sep='')
    sum_all = 0
    for j in ldf[i]:
        sum_all += ldf[i][j]
    for j in sorted(ldf[i], key=ldf[i].get, reverse=True):
        print(j, ' составляет ', round(ldf[i][j]/sum_all, 3), '% (', ldf[i][j], ')', sep='')

Сделайте то же самое для определителя на символьных энграммах.

In [ ]:
lang_detects_ngrams = {}

for i in new_corpora:
    lang_detects_ngrams[i] = Counter(clf.predict(new_corpora[i]))

In [ ]:
ldn = {}

for a, b in lang_detects_ngrams.items():
    ldn[a] = {}
    
for key, value in lang_detects_ngrams.items():
    if 'ru' in value:
        ldn[key]['русский'] = lang_detects_ngrams[key]['ru']
    if 'uk' in value:
        ldn[key]['украинский'] = lang_detects_ngrams[key]['uk']
    if 'en' in value:
        ldn[key]['английский'] = lang_detects_ngrams[key]['en']
    if 'err' in value:
        ldn[key]['ошибка'] = lang_detects_ngrams[key]['err']
    if 'bxr' in value:
        ldn[key]['бурятский'] = lang_detects_ngrams[key]['bxr']

In [ ]:
for i in ldn:
    print('\nВ паблике ', i, ':', sep='')
    sum_all = 0
    for j in ldn[i]:
        sum_all += ldn[i][j]
    for j in sorted(ldn[i], key=ldn[i].get, reverse=True):
        print(j, ' составляет ', round(ldn[i][j]/sum_all, 3), '% (', ldn[i][j], ')', sep='')

Обсудите работу каждого из классификаторов, обсудите ошибки, объясните разницу в результатах.

Частотные слова:
Очень радует соотношение яызков в русском и английском пабликах, так как оно, видимо, близко к реальности. Для украинских, к сожалению, классификатор сильно путается между русским и ураинским, наверно, из-за сходства языков и того, что, возможно, в этих пабликах многие пишут и по-русски тоже. По крайней мере, в ukrlit_memes, русскоязычных комментариев больше, чем в ukrainer_net. С бурятским у алгоритма все совсем плохо, слишком большой процент нераспознанных комментариев и, скорее всего, бурятских, принятых за русские. Но в этих пабликах тоже, видимо, много действительно русской речи. В целом, алгоритм работает неплохо, но можно и лучше, как мы видим из второго классификатора.


N-grams:
Радует уже то, что этот классификатор не может не распознать язык, он его все равно определяет к какому-то классу. Гораздо лучше выглядит статистика для украинских пабликов и для бурятских. Все равно присутствуют какие-то сомнительные "бурятские" комментарии во всех пабликах, а также какой-то процент "английских" в украинских и бурятских.


В целом:
Второй классификатор кажется гораздо более надежным. Скорее всего, это связано с тем, что комментарии достаточно короткие (как бы мы их не очищали), а также зачастую состоят из довольно несвойственных для википедии слов (какие-нибудь эмоциональные "лол" или ругательства, присущие каждому языку). Довольно странно судить по частотным для полуофициального источника словам комментарии в соц сети. Может, просто стоило взять частотный список подлиннее. N-граммы справились с задачей получше, но по небольшим ошибкам видно, что в алгоритме все же были какие-то "выбросы", возможно, связанные с неподходящей степенью чистки корпуса (смайлики, например). Но во втором классификаторе такие ошибки составляют слшком маленькю долю в соотношении, так что их можно списывать на помехи (или там действительно есть комментарии на соответствующих языках). Надежнее было бы проверить точность нашего алгоритма на какой-то выборке комментариев с уже определенным языком, с ответом, но у нас нет такой возможности.